In [66]:
from __future__ import print_function, division
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("spark://10.211.55.100:7077") \
   .appName("test") \
   .enableHiveSupport() \
   .getOrCreate()

sc = spark.sparkContext

# 共享变數(shared varable)

### 變數在Driver 上的更新不會傳回 Master
一般情况，当传递一个操作函数( 例如 map 或 reduce) 给 Spark时，Spark 实际上是操作这个函数变数的副本。这些变数被复制到每台机器上，而且这些变数在远端机器上的更新都不会传送回驱动程式。一般来说，跨任务的读写操作变数效率不高而Driver是我们提交Spark程式的节点，并且所有的reduce类型的操作都会汇总到Driver节点进行整合。节点之间会将map/reduce等操作函数传递一个独立副本到每一个节点，这些变数也会复制到每台机器上，而节点之间的运算是相互独立的，变数的更新并不会传递回Driver程式。 

### 共享變數能讓節點之間共享同個值
那么有个问题，如果我们想在节点之间共享一份变数，比如一份公共的配置项， Spark为我们提供了两种特定的共享变数，来完成节点间变数的共享广播变数 ( broadcast variable) 与累加器 ( accumulator)



## 广播变数 (broadcast)
广播变数允许程式将一个可读变数存在每台机器的记忆体里，而不是每个任务都存有一份副本。例如，利用广播变数，我们能将一个大资料量输入的集合副本发送到每个节点上。

累加器比较简单直观，如果我们需要在spark中进行一些全局统计就可以使用它。但是有时候仅仅一个累加器并不能满足我们的需求，比如资料库中一份公共配置表格，需要同步给各个节点进行查询。Spark也尝试着利用有效的广播演算法去分配广播变数，以减少通信的成本。

### 建立 RDD

In [70]:
rdd  = sc.parallelize(['dog', 'cat', 'dog', 'cat', 'cat'],4)

In [104]:
mapper = {'dog': 1 ,'cat': 2}
mapper['dog']

1

In [105]:
broadcastVar = sc.broadcast(mapper)

In [106]:
broadcastVar.value

{'cat': 2, 'dog': 1}

In [75]:
broadcastVar.value.get('cat')

2

In [80]:
rdd.map(lambda x: broadcastVar.value.get(x)).collect()

[1, 2, 1, 2, 2]

## 更新廣播變項
广播变量创建以后，我们就能够在集群的任何函数中使用它来代替变量v，这样我们就不需要再次传递变量v到每个节点上。另外，为了保证所有的节点得到广播变量具有相同的值，对象v不能在广播之后被修改。

In [111]:
broadcastVar['pig']  = 3

TypeError: 'Broadcast' object does not support item assignment

In [96]:
broadcastVar.unpersist()

In [107]:
mapper['pig'] = 3

In [108]:
mapper

{'cat': 2, 'dog': 1, 'pig': 3}

In [119]:
broadcastVar = sc.broadcast(mapper)

In [120]:
broadcastVar.value

{'cat': 2, 'dog': 1, 'pig': 3}

### 解除廣播變項

In [121]:
broadcastVar.destroy()

In [122]:
broadcastVar.value

{'cat': 2, 'dog': 1, 'pig': 3}

In [123]:
rdd.map(lambda x: broadcastVar.value.get(x)).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Task not serializable
	at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:298)
	at org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:288)
	at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:108)
	at org.apache.spark.SparkContext.clean(SparkContext.scala:2101)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1950)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1965)
	at org.apache.spark.rdd.RDD$$anonfun$collect$1.apply(RDD.scala:936)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:362)
	at org.apache.spark.rdd.RDD.collect(RDD.scala:935)
	at org.apache.spark.api.pyt

In [124]:
broadcastVar.destroy()

Py4JJavaError: An error occurred while calling o668.destroy.
: org.apache.spark.SparkException: Attempted to use Broadcast(14) after it was destroyed (destroy at NativeMethodAccessorImpl.java:0) 
	at org.apache.spark.broadcast.Broadcast.assertValid(Broadcast.scala:144)
	at org.apache.spark.broadcast.Broadcast.destroy(Broadcast.scala:107)
	at org.apache.spark.broadcast.Broadcast.destroy(Broadcast.scala:98)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.j